# Imports

In [ ]:
from os.path import join
from autocanvas.pipes.ufsoc import (get_sections_from_ufsoc, 
                                    store_section_ta_to_csv)
from autocanvas.config import INPUT_DIR
from autocanvas.core import Periods, day_abbrs

import pandas as pd
import datetime
pd.set_option('display.max_columns', 500)

# Tests

In [ ]:
df = get_sections_from_ufsoc(semester="Fall 2020",
                             course_code="PHY2054",
                             course_title="Physics 2",
                             instructor="Weatherford",
                             department="Physics",
#                             headless=False
                            )
df

# UF Schedule of courses

ChromeDriver and Selenium need to be installed and be compatible with the installed Chrome/Chromium version. To decide on the version of `chromedriver`, check first the Chrome version. After downloading the appropriate version of `chromedriver`, I prefer to put the `chromedriver` executable in `/usr/local/share`. For selenium, just use `pip`.

In [ ]:
course_season = "Fall"
year = "2021"
instructor_last_name = "Yelton"
course_code = "PHY2048"

semester = course_season + " " + year
query_args = dict(
    semester=semester,
    course_code=course_code,
    course_title="",
    instructor=instructor_last_name,
    department="Physics",
)
# the stored output of this function will be used as 
# input for other routines
identifier = (course_code + "_" + course_season + year).lower()
print(identifier)
df_sections = store_section_ta_to_csv(output_dir=INPUT_DIR, 
                        identifier=identifier, 
                        headless=True,
                        **query_args)
df_sections

In [ ]:
# ids = driver.find_elements_by_xpath('//*[@id]')
# for ii in ids:
#     #print ii.tag_name
#     print(ii.get_attribute('id'))

# Quiz Times Pipeline

In [ ]:
date_of_first_monday = pd.Timestamp("Aug 23 2021")

quiz_to_week = {
    2:4,
    3:5,
    4:6,
    5:7,
    6:8,
    7:9,
    8:10,
    9:11,
    10:13,
    11:14,
    12:16,
}

quiz_day = {
    "M":"M",
    "T":"T",
}

# quiz_day = {
#     "MW":"M",
#     "TR":"T",
#     "WF":"W"
# }

# how many hh:mm:ss after the start of the recitation session does the quiz start
quiz_time_offset_from_start = "00:28:00"
quiz_availability_window = "00:32:00"

In [ ]:
df = df_sections["meeting_2"].str.split('|', expand=True)
df.columns = ["Days", "Period"]

df["Days"] = (df["Days"]
              .str.replace(',', '')             
              .str.strip()
             )

df["Period"] = df["Period"].str.extract('(\d+)')
df["dayperiod"] = df.agg(' '.join, axis=1)
df["Period"] = df["Period"].astype(int)

df = pd.merge(left=df_sections,
              right=df, 
              left_index=True,
              right_index=True)

df_periods = Periods.from_season(course_season).get_dataframe()

df["discussion_start_time"] = (df["Period"]
                               .map(lambda item:
                                    df_periods["start_times"][item])
                              )  

df["discussion_end_time"] = (df["Period"]
                             .map(lambda item:
                                  df_periods["end_times"][item])
                            )

df["quiz_day"] = df["Days"].map(quiz_day)

for quiz_number, week_number in quiz_to_week.items():
    column_from = "Quiz {} start time".format(quiz_number)
    column_until = "Quiz {} due time".format(quiz_number)
    # add 7 days to the date for each week past Week 1
    # and 1 day for each day after Monday
    quiz_date = (date_of_first_monday 
                 + (week_number-1) * pd.to_timedelta("7D") 
                 + pd.to_timedelta(df["quiz_day"]
                                   .map(lambda item:
                                        "{}D".format(day_abbrs.index(item))
                                       )
                                  )
                ).astype(str)
    
    # combine date and wall-time
    discussion_datetime_start = pd.to_datetime(quiz_date \
                                + " " \
                                + df["discussion_start_time"].astype(str)
                                )
    # add offset between discussion session start times and quiz start times  
    df[column_from] = discussion_datetime_start + \
                        pd.to_timedelta(quiz_time_offset_from_start)
    df[column_until] = df[column_from] + \
                        pd.to_timedelta(quiz_availability_window)

output_path = join(INPUT_DIR, "sections_"+identifier+".xlsx")
df.to_excel(output_path, index=False)
    
df